# Hybrid Plant with Offshore Wind, Solar, Battery and Hydrogen Example
---
In this example we will simulate an offshore hybrid plant using the GreenHEART software. This software can be configured to simulate both onshore and offshore plant scenarios.

This simulation starts with renewable energy generation using offshore wind and solar PV technologies, which then can be stored in a battery or directly supplied to a PEM electrolyzer for hydrogen production. If the hydrogen is produced offshore it can be transport onshore for use or storage or be stored offshore.

### Import Required Modules
---
Begin by importing the necessary modules for the GreenHEART simulation.

In [1]:
#general imports
import os

# GreenHEART imports
from greenheart.simulation.greenheart_simulation import GreenHeartSimulationConfig
from greenheart.tools.optimization.gc_run_greenheart import run_greenheart

# ORBIT imports
from ORBIT.core.library import initialize_library
initialize_library(os.path.join(os.getcwd(), "./inputs/10-inputs/"))

/Users/kbrunik/github/HOPP/examples/log/hybrid_systems_2024-07-16T20.36.10.728977.log


/Users/kbrunik/anaconda3/envs/gh/lib/python3.8/site-packages/fastkml/config.py:28: UserWarning: Package `lxml` missing. Pretty print will be disabled
  warnings.warn("Package `lxml` missing. Pretty print will be disabled")


ORBIT library intialized at '/Users/kbrunik/github/HOPP/examples/./inputs/10-inputs/'


### Load Simulation Input Files
---
Input files are in a YAML file format. Five YAML files are required for offshore analysis:
1. HOPP Configuration YAML
2. GreenHEART Configuration YAML
3. Wind Turbine Configuration YAML
4. FLORIS Configuration YAML
5. ORBIT Configuration YAML

#### HOPP YAML Overview

The HOPP YAML file includes information required for simulating the renewable energy generation at the plant. Such as:

- **Analysis Location**: Specifies latitude, longitude, site boundaries, and resource files.
- **Generation Technologies**: Encompasses details on wind, solar PV, and battery storage systems.
- **General Generation Configuration Options**: Encompasses simulation options, battery dispatch strategies, and cost information.

In [9]:
filename_hopp_config = "./inputs/10-inputs/plant/hopp_config_gom.yaml"

#### GreenHEART YAML Overview

The GreenHEART YAML file includes information required for simulating the renewable energy generation through the end product production (hydrogen, steel, ammonia). Such as:

- **Plant Design Scenarios**: Specifies location and connection of individual components of plant.
- **Financial Parameters**: General financial parameters needed for calculating the levelized cost of product and other financial metrics.
- **Hydrogen Equipment Information**: Including electrolyzer specifications, compressor, pipeline, and H<sub>2</sub> storage information.
- **Policy Options**: Describes options to include [IRA](https://www.whitehouse.gov/wp-content/uploads/2022/12/Inflation-Reduction-Act-Guidebook.pdf) tax incentives policy in GreenHEART simulation.

In [3]:
filename_greenheart_config = "./inputs/10-inputs/plant/greenheart_config_offshore_gom.yaml"

#### Wind Turbine and FLORIS YAML Overview

The GreenHEART simulation runs a steady-state wake modeling tool ([FLORIS](https://www.nrel.gov/wind/floris.html)) to assess the wind farm's energy production. For accurate modeling within FLORIS, two configuration YAML files are required:

- The **Wind Turbine YAML** file includes the description of the turbine including turbine rating rotor diameter, hub hight, and the power curve.
- The **FLORIS YAML** file includes atmospheric conditions and wake models used in simulation.

In [4]:
turbine_model = "osw_17MW"
filename_turbine_config = "./inputs/10-inputs/turbines/" + turbine_model + ".yaml"
filename_floris_config = "./inputs/10-inputs/floris/floris_input_" + turbine_model + ".yaml"

#### ORBIT YAML Overview

The GreenHEART simulation uses [ORBIT](https://www.nrel.gov/docs/fy20osti/77081.pdf) to calculate offshore wind component, installation and balance-of-system costs. ORBIT requires a YAML file which includes:
- **Additional Site Information**: Used for parameterizing the offshore environment.
- **Equipment Information**: Includes specifications for cabling, substructure design, etc.
- **Vessel Information**: Information about vessels used to install and maintain offshore plant.
- **Installation Information**: Installation availability and installation phases.

In [5]:
filename_orbit_config= "./inputs/10-inputs/plant/orbit-config-"+turbine_model+"-gom.yaml"

### Create Full Configuration to Run GreenHEART Simulation
---
The class `GreenHeartSimulationConfig` reads in all the required YAML files described above, allows user to set additional settings while running the GreenHEART simulation, these include:
- **verbose**: If set to `True` additional information about the simulation will be printed as it runs.
- **show_plots**: If set to `True` the plots will pop up during the simulation.
- **save_plots**: If set to `True` the plots will be saved in a figures folder.
- **use_profast**: [ProFAST](https://github.com/NREL/ProFAST) is a financial analysis software that provides a quick and convenient in-depth financial analysis for production systems and services based on Generally Accepted Accounting Principles (GAAP) methodology.
- **incentive_options**: Can select IRA incentives options which are located in the `GreenHEART YAML`.
- **plant_design_scenario**: Can select the plant design scenario from options which are located in the `GreenHEART YAML`.
- **output_level**: Determines what information is returned from the simulation.

In [17]:
config = GreenHeartSimulationConfig(
    filename_hopp_config=filename_hopp_config,
    filename_greenheart_config=filename_greenheart_config,
    filename_turbine_config=filename_turbine_config,
    filename_orbit_config=filename_orbit_config,
    filename_floris_config=filename_floris_config,
    verbose=True,
    show_plots=False,
    save_plots=False,
    use_profast=True,
    post_processing=False,
    incentive_option=1,
    plant_design_scenario=1,
    output_level=5,
)


Plant configuration:
turbine :  osw_17MW
installation_availability :  0.7
site :  {'area': 142, 'depth': 45, 'distance': 141.5414, 'distance_to_landfall': 44.74089, 'mean_windspeed': False}
plant :  {'capacity': 986, 'layout': 'grid', 'num_turbines': 58, 'row_spacing': 7, 'substation_distance': 1, 'turbine_spacing': 7}
landfall :  {'interconnection_distance': 5, 'trench_length': 4}
array_cable_install_vessel :  example_cable_lay_vessel
export_cable_install_vessel :  example_cable_lay_vessel
oss_install_vessel :  example_heavy_lift_vessel
spi_vessel :  example_scour_protection_vessel
wtiv :  example_wtiv
OffshoreSubstationInstallation :  {'feeder': 'example_heavy_feeder', 'num_feeders': 1}
array_system :  {'free_cable_length': 0.5}
array_system_design :  {'cables': ['XLPE_185mm_66KV', 'XLPE_630mm_66KV']}
export_system_design :  {'cables': ['XLPE_1000m_220kV'], 'percent_added_length': 0.1}
scour_protection_design :  {'cost_per_tonne': 40, 'scour_protection_depth': 1}
MonopileDesign :  {

### Run the GreenHEART Simulation
---
The simulation runs a techno-economic analysis of a given plant configuration. Data and figures are output to a seperate folder.

In [18]:
# for analysis
prob, config = run_greenheart(config, run_only=True)

# for optimization
# prob, config = run_greenheart(config, run_only=False)

/Users/kbrunik/anaconda3/envs/gh/lib/python3.8/site-packages/openmdao/core/driver.py:546: OpenMDAOWarning:Driver: No matches for pattern '*_df' in recording_options['excludes'].


Installation CapEx:  284 M
System CapEx:        383 M
Turbine CapEx:       1479 M
Soft CapEx:          636 M
Total CapEx:        2933 M
Annual OpEx Rate:        52510889 

Installation Time: 24397 h

N Substations:  2
N cables:  4


Cable specific costs
Export cable installation CAPEX: 71.71 M USD


FLORIS is the system model...
Simulating wind farm output in FLORIS...

HOPP Results
Hybrid Annual Energy:  {"pv": 1851330.6142037676, "wind": 3755361454.261004, "battery": 75138.86048135585, "hybrid": 3757287923.7356806}
Capacity factors:  {"pv": 27.474084457361847, "wind": 43.47811662661976, "battery": 0, "hybrid": 39.46690193295024}
Real LCOE from HOPP:  {"pv": null, "wind": 0, "battery": null, "hybrid": null}
Took 0.428 sec to run the create clusters
Took 0.018 sec to run even_split_power function
Took 0.37 sec to run the RUN function

Electrolyzer Physics:
H2 Produced Annually (tonnes):  67061.82816844688
Max H2 hourly (tonnes):  16.845396595785363
Max H2 daily (tonnes):  374.528427299

### Key Output Values

In [19]:
lcoe = prob.get_val("lcoe", units="USD/(MW*h)")
lcoh = prob.get_val("lcoh", units="USD/kg")

print("LCOE: ", lcoe, "[$/MWh]")
print("LCOH: ", lcoh, "[$/kg]")

LCOE:  [147.74611336] [$/MWh]
LCOH:  [7.97388197] [$/kg]
